In [1]:
#download dependencies
!wget -P . https://raw.githubusercontent.com/quietscientist/gma_score_prediction_from_video/refs/heads/main/utils/kinematics.py
!wget -P . https://raw.githubusercontent.com/quietscientist/gma_score_prediction_from_video/refs/heads/main/utils/circstat.py
!wget -P . https://raw.githubusercontent.com/quietscientist/gma_score_prediction_from_video/refs/heads/main/utils/processing.py
!wget -P . https://raw.githubusercontent.com/quietscientist/gma_score_prediction_from_video/refs/heads/main/utils/skeleton.py

#download example data or upload your own json annotations

import os
os.makedirs('./data',exist_ok=True)
!wget -P . https://figshare.com/ndownloader/articles/25316500/versions/1

#unzip data into ./data folder and remove zip file
!unzip ./1 -d ./data
!rm ./1

--2024-12-05 17:48:56--  https://raw.githubusercontent.com/quietscientist/gma_score_prediction_from_video/refs/heads/main/utils/kinematics.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9865 (9.6K) [text/plain]
Saving to: ‘./kinematics.py’

kinematics.py       100%[===================>]   9.63K  --.-KB/s    in 0s      

2024-12-05 17:48:56 (93.0 MB/s) - ‘./kinematics.py’ saved [9865/9865]

--2024-12-05 17:48:56--  https://raw.githubusercontent.com/quietscientist/gma_score_prediction_from_video/refs/heads/main/utils/circstat.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP r

In [2]:
%pip install scikit-video

import sys, os, cv2, glob, json, gc
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation
import json
import numpy as np
import pandas as pd
import itertools
from itertools import chain
from moviepy.editor import VideoFileClip
import skvideo.io
from tqdm import tqdm
import circstat as CS
import scipy as sc
import math

from processing import *
from kinematics import *
from skeleton import *

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 18.9 MB/s eta 0:00:00


  if event.key is 'enter':



In [3]:
# format files as pkl with openpose standard and bodypart labels

from tqdm import tqdm

OVERWRITE = True
USE_CENTER_INSTANCE = False
USE_BEST_INSTANCE = True

dataset = 'Youtube'
json_path = f'/content/data/Infant Pose Data/{dataset}/annotations'
json_files = os.listdir(json_path)
directory = f'./data'

save_path = f'./pose_estimates/{dataset}_norm'

if not os.path.exists(save_path):
    os.makedirs(save_path)

kp_mapping = {0:'Nose', 1:'Neck', 2:'RShoulder', 3:'RElbow', 4:'RWrist', 5:'LShoulder', 6:'LElbow',
              7:'LWrist', 8:'RHip', 9:'RKnee', 10:'RAnkle', 11:'LHip',
              12:'LKnee', 13:'LAnkle', 14:'REye', 15:'LEye', 16:'REar', 17:'LEar'}

# Define the DataFrame columns as specified
columns = ['video_number', 'video', 'bp', 'frame', 'x', 'y', 'c','fps', 'pixel_x', 'pixel_y', 'time', 'part_idx']
data = []  # This will hold the data to be loaded into the DataFrame

if dataset == 'Youtube':
    vid_info = pd.read_csv('/content/data/YT_video_info.csv')
elif dataset == 'Clinical':
    vid_info = pd.read_csv('/content/data/CLIN_video_info.csv')


for file_number, file in enumerate(tqdm(json_files)):
    # Construct the full file path
    file_path = os.path.join(json_path, file)
    fname = file.split('.')[0]
    interim = []

    if not OVERWRITE and os.path.exists(f'{save_path}/{fname}.pkl'):
        continue

    # Open and load the JSON data
    with open(file_path, 'r') as f:
        frames = json.load(f)
        info = vid_info[vid_info['video'] == fname]
        center_x = info['center_x'].values[0]
        center_y = info['center_y'].values[0]
        pixel_x = info['width'].values[0]
        pixel_y = info['height'].values[0]
        fps = info['fps'].values[0]

        # Iterate through each frame in the JSON file
        for frame in frames:
            frame_id = frame['frame_id']
            if 'instances' in frame and len(frame['instances']) > 0:

                if USE_CENTER_INSTANCE:
                    instance_id = get_center_instance(frame['instances'], center_x, center_y)
                elif USE_BEST_INSTANCE:
                    instance_id = get_best_instance(frame['instances'])
                else:
                    instance_id = 0

                keypoints = frame['instances'][instance_id]['keypoints']
                confidence = frame['instances'][instance_id]['keypoint_scores']
                keypoints, confidence = convert_coco_to_openpose(keypoints, confidence)

                # Iterate through each keypoint
                for part_idx, (x, y) in enumerate(keypoints):

                    bp = kp_mapping[part_idx]
                    fps = fps
                    time = frame_id / fps
                    c = confidence[part_idx]

                    row = [file_number, fname, bp, frame_id, x, y, c, fps, pixel_x, pixel_y, time, part_idx]
                    interim.append(row)

    interim_df = pd.DataFrame(interim, columns=columns)
    interim_df.to_pickle(f'{save_path}/{fname}.pkl')

    del interim_df


100%|██████████| 75/75 [00:17<00:00,  4.31it/s]


In [4]:
for pklfile in tqdm(os.listdir(save_path)):

    interim_df = pd.read_pickle(f'{save_path}/{pklfile}')
    interim_df.to_csv(f'{save_path}/pose_estimates_{dataset}.csv', mode='a', header=False, index=False)

    del interim_df

100%|██████████| 75/75 [00:18<00:00,  4.03it/s]


In [6]:
dataset = 'Youtube'

csv_path = f'{save_path}/pose_estimates_{dataset}.csv'
output_csv_path = f'{save_path}/pose_estimates_{dataset}_b.csv'
chunksize = 1000  # Number of rows per chunk

# Define the new headers
new_headers = ['video_number', 'video', 'bp', 'frame', 'x', 'y', 'c', 'fps', 'pixel_x', 'pixel_y', 'time', 'part_idx']

# Read the CSV file in chunks
chunk_iterator = pd.read_csv(csv_path, chunksize=chunksize)

# Process the first chunk
first_chunk = next(chunk_iterator)
first_chunk.columns = new_headers
first_chunk.to_csv(output_csv_path, index=False)

# Process the rest of the chunks and append them to the new CSV file without headers
for chunk in chunk_iterator:
    chunk.columns = new_headers
    chunk.to_csv(output_csv_path, mode='a', index=False, header=False)

# rename the csv file
os.rename(csv_path, f'{save_path}/pose_estimates_{dataset}_x.csv')
os.rename(output_csv_path, csv_path)


In [45]:
def process_dataframe(df, pose_estimate_path):
    if df.empty:
        print("DataFrame is empty, skipping processing.")
        return

    print(f"Processing DataFrame for video_number: {df['video_number'].iloc[0]}")
    try:
        session = df['video'].unique()[0].split('_')[1][0]
        infant = df['video'].unique()[0].split('_')[1][3:]
        age = 'month'

        print(f'infant: {infant}, session: {session}')

    except:
        f'could not process video {df["video"].unique()[0]}'
        return

    median_window = 1
    mean_window = 1
    delta_window = 0.25  # Smoothing applied to delta_x, velocity, acceleration

    df['x'] = pd.to_numeric(df['x'])
    df['y'] = pd.to_numeric(df['y'])

    # Interpolate
    df = df.groupby(['video', 'bp']).apply(interpolate_df).reset_index(drop=True)

    # Median and mean filter
    median_window = 0.5
    mean_window = 0.5
    df = df.groupby(['video', 'bp']).apply(lambda x: smooth(x, 'y', median_window, mean_window)).reset_index(drop=True)
    df = df.groupby(['video', 'bp']).apply(lambda x: smooth(x, 'x', median_window, mean_window)).reset_index(drop=True)

    try:
        # Rotate and normalise by reference
        xdf = normalise_skeletons(df)
        xdf = get_dynamics_xy(xdf, delta_window)
        xdf.to_pickle(f'{pose_estimate_path}/xdf/{infant}_{session}_processed_pose_estimates_coords_norm.pkl')

        adf = get_joint_angles(df)
        adf = get_dynamics_angle(adf, delta_window)
        adf.to_pickle(f'{pose_estimate_path}/adf/{infant}_{session}_processed_pose_estimates_coords_norm.pkl')

    except KeyError as e:
        print(f'Error processing video for {infant}: {e}')


In [19]:
# Smooth detections and compute features

pose_estimate_path = f'./pose_estimates/{dataset}_norm'
csv_path = f'{pose_estimate_path}/pose_estimates_{dataset}.csv'
save_path = f'{pose_estimate_path}/pose_estimates_{dataset}_processed.csv'

# Create necessary directories
os.makedirs(pose_estimate_path, exist_ok=True)
os.makedirs(f'{pose_estimate_path}/xdf', exist_ok=True)
os.makedirs(f'{pose_estimate_path}/adf', exist_ok=True)

In [44]:
chunksize = 100000
buffer = pd.DataFrame()

# Read the CSV file in chunks
chunk_iterator = pd.read_csv(csv_path, chunksize=chunksize)

for chunk in chunk_iterator:
  unique_videos = chunk['video_number'].unique()

  for video_number in unique_videos:
      video_chunk = chunk[chunk['video_number'] == video_number]

      if not buffer.empty:
        if buffer['video_number'].iloc[0] == video_number:

            buffer = pd.concat([buffer, video_chunk], ignore_index=True)
            if video_number not in chunk['video_number'].values:
                process_dataframe(buffer, pose_estimate_path)
                buffer = pd.DataFrame()

        else:
            process_dataframe(buffer, pose_estimate_path)
            buffer = video_chunk
      else:
          buffer = video_chunk

  chunk = chunk[~chunk['video_number'].isin(unique_videos)]

# # Process any remaining rows in the buffer
# if not buffer.empty:
#     print("Processing remaining rows in the buffer...")
#     process_dataframe(buffer, pose_estimate_path)

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['x'] = pd.to_numeric(df['x'])

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['y'] = pd.to_numeric(df['y'])

  df = df.groupby(['video', 'bp']).apply(interpolate_df).reset_index(drop=True)

  df = df.groupby(['video', 'bp']).apply(lambda x: smooth(x, 'y', median_window, mean_window)).reset_index(drop=True)



Processing DataFrame for video_number: 13
0
179
infant: 179, session: 0


  df = df.groupby(['video', 'bp']).apply(lambda x: smooth(x, 'x', median_window, mean_window)).reset_index(drop=True)

  s_angle = pd.merge(s_angle, uref, on=['video', 'frame'], how='inner')

  s_angle = pd.merge(s_angle, lref, on=['video', 'frame'], how='inner')



   video_number         video      bp  frame   x   y         c    fps  \
0            13  video_000179  LAnkle      0 NaN NaN  0.893593  29.97   
1            13  video_000179    LEar      0 NaN NaN  0.955352  29.97   
2            13  video_000179  LElbow      0 NaN NaN  0.930716  29.97   
3            13  video_000179    LEye      0 NaN NaN  0.984404  29.97   
4            13  video_000179    LHip      0 NaN NaN  0.874981  29.97   

   pixel_x  pixel_y  ...  urefx  lrefy  lrefx  refx  refy  ref_dist  \
0   1280.0    720.0  ...    NaN    NaN    NaN   NaN   NaN       NaN   
1   1280.0    720.0  ...    NaN    NaN    NaN   NaN   NaN       NaN   
2   1280.0    720.0  ...    NaN    NaN    NaN   NaN   NaN       NaN   
3   1280.0    720.0  ...    NaN    NaN    NaN   NaN   NaN       NaN   
4   1280.0    720.0  ...    NaN    NaN    NaN   NaN   NaN       NaN   

   ref_angle  x_rotate  y_rotate   delta_t  
0        NaN       NaN       NaN  0.033367  
1        NaN       NaN       NaN  0.033367  

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['delta_t'] = 1/df['fps']

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['x'] = pd.to_numeric(df['x'])

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['y'] = pd.to_numeric(df['y'])

  df = df.groupby(['video', 'bp']).apply(interpolate_df).reset_index(drop=True)

  df = df.groupby(['video',

...

Processing DataFrame for video_number: 8
0
297
infant: 297, session: 0


  df = df.groupby(['video', 'bp']).apply(lambda x: smooth(x, 'x', median_window, mean_window)).reset_index(drop=True)

  s_angle = pd.merge(s_angle, uref, on=['video', 'frame'], how='inner')

  s_angle = pd.merge(s_angle, lref, on=['video', 'frame'], how='inner')



   video_number         video      bp  frame   x   y         c   fps  pixel_x  \
0             8  video_000297  LAnkle      0 NaN NaN  0.921482  30.0    270.0   
1             8  video_000297    LEar      0 NaN NaN  0.950550  30.0    270.0   
2             8  video_000297  LElbow      0 NaN NaN  0.896717  30.0    270.0   
3             8  video_000297    LEye      0 NaN NaN  0.938264  30.0    270.0   
4             8  video_000297    LHip      0 NaN NaN  0.826959  30.0    270.0   

   pixel_y  ...  urefx  lrefy  lrefx  refx  refy  ref_dist  ref_angle  \
0    360.0  ...    NaN    NaN    NaN   NaN   NaN       NaN        NaN   
1    360.0  ...    NaN    NaN    NaN   NaN   NaN       NaN        NaN   
2    360.0  ...    NaN    NaN    NaN   NaN   NaN       NaN        NaN   
3    360.0  ...    NaN    NaN    NaN   NaN   NaN       NaN        NaN   
4    360.0  ...    NaN    NaN    NaN   NaN   NaN       NaN        NaN   

   x_rotate  y_rotate   delta_t  
0       NaN       NaN  0.033333  
1     

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['delta_t'] = 1/df['fps']



KeyboardInterrupt: 

In [ ]:
# Compute angular features

HEADER_WRITTEN = False

for file in tqdm(os.listdir(f'{pose_estimate_path}/adf')):
    if 'pkl' in file:

        adf = pd.read_pickle(os.path.join(f'{pose_estimate_path}/adf', file))

        # angle features
        feature_angle = adf.groupby(['bp','video']).apply(angle_features).reset_index(drop=True)
        feature_angle = pd.pivot_table(feature_angle, index='video', columns=['bp'])
        l0 = feature_angle.columns.get_level_values(1)
        l1 = feature_angle.columns.get_level_values(0)
        cols = [l1[i]+'_'+l0[i] for i in range(len(l1))]
        feature_angle.columns = cols
        feature_angle =feature_angle.reset_index()

        # - measure of symmetry (left-right cross correlation)
        corr_joint = adf.groupby(['video', 'part']).apply(lambda x:corr_lr(x,'angle')).reset_index()
        corr_joint['part'] = 'lrCorr_angle_'+corr_joint['part']
        corr_joint.columns = ['video', 'feature', 'Value']
        corr_joint = pd.pivot_table(corr_joint, index='video', columns=['feature'])
        l1 = corr_joint.columns.get_level_values(1)
        corr_joint.columns = l1
        corr_joint = corr_joint.reset_index()
        feature_angle = pd.merge(feature_angle,corr_joint, on='video', how='outer')

        if not HEADER_WRITTEN:
            feature_angle.to_csv(f'{pose_estimate_path}/features_angle.csv', header=True, index=False)
            HEADER_WRITTEN = True
        else:
            feature_angle.to_csv(f'{pose_estimate_path}/features_angle.csv', header=False, index=False, mode='a')

In [ ]:
# Compute XY features

HEADER_WRITTEN = False

for file in tqdm(os.listdir(f'{pose_estimate_path}/xdf')):
    if 'pkl' in file:

        xdf = pd.read_pickle(os.path.join(f'{pose_estimate_path}/xdf', file))

        # xy features
        bps = ['LAnkle', 'RAnkle', 'LWrist', 'RWrist']
        feature_xy = xdf[np.isin(xdf.bp, bps)].groupby(['bp','video']).apply(xy_features).reset_index(drop=True)
        feature_xy = pd.pivot_table(feature_xy, index='video', columns=['bp'])
        l0 = feature_xy.columns.get_level_values(1)
        l1 = feature_xy.columns.get_level_values(0)
        cols = [l1[i]+'_'+l0[i] for i in range(len(l1))]
        feature_xy.columns = cols
        feature_xy = feature_xy.reset_index()
        # - measure of symmetry (left-right cross correlation)
        xdf['dist'] = np.sqrt(xdf['x']**2+xdf['y']**2)
        corr_joint = xdf.groupby(['video', 'part']).apply(lambda x:corr_lr(x,'dist')).reset_index()
        corr_joint['part'] = 'lrCorr_x_'+corr_joint['part']
        corr_joint.columns = ['video', 'feature', 'Value']
        corr_joint = pd.pivot_table(corr_joint, index='video', columns=['feature'])
        l1 = corr_joint.columns.get_level_values(1)
        corr_joint.columns = l1
        corr_joint = corr_joint.reset_index()
        feature_xy = pd.merge(feature_xy, corr_joint, on='video', how='outer')

        if not HEADER_WRITTEN:
            feature_xy.to_csv(f'{pose_estimate_path}/features_xy.csv', header=True, index=False, mode='a')
            HEADER_WRITTEN = True
        else:
            feature_xy.to_csv(f'{pose_estimate_path}/features_xy.csv', header=False, index=False, mode='a')

In [ ]:
# Combine features

SAVE = True

if features_xy.empty or features_angle.empty:
  features_xy = pd.read_csv(f'{pose_estimate_path}/features_xy.csv')
  features_angle = pd.read_csv(f'{pose_estimate_path}/features_angle.csv')

features = pd.merge(features_xy, features_angle, on='video', how='inner')

In [ ]:
# Split features based on video file naming convention

features['infant'] = features['video'].str.split('_').str[0]
features['session'] = features['video'].str.split('_').str[1]
features['age'] = features['video'].str.split('_').str[6]

if SAVE:
    features.to_csv(f'{pose_estimate_path}/features.csv', header=True, index=False)

In [ ]:
# Include data for the following, and restructure dataframe
# Customize for your specific dataset

id_vars = ['infant', 'age','corr_age', 'chrono_age', 'score','session','video']
melted = pd.melt(features, id_vars=id_vars, var_name="feature", value_name="Value")


In [ ]:
feature_str = melted.feature.str.split('_')

side =pd.Series([ i[-1:][0][0] if i[0]!='lrCorr' else '' for i in feature_str])
part =pd.Series([ i[-1:][0][1:] if i[0]!='lrCorr' else i[-1:][0] for i in feature_str])
feature = pd.Series(['_'.join(i[:-1]) for i in feature_str])

feature_attributes = pd.DataFrame.from_dict({'side': side, 'part': part, 'feature_name': feature})

feature_attributes.feature_name.unique()
melted[['side', 'part', 'feature']] = feature_attributes

melted = melted.dropnans()
mean = melted.groupby(['infant', 'age', 'corr_age', 'chrono_age', 'score', 'session', 'video', 'feature', 'part'])['Value'].mean().reset_index()
mean.to_csv(f'{pose_estimate_path}/{dataset}_features_mean_by_side.csv', header=True, index=False)
